In [68]:
import numpy as np
import matplotlib as mplt
import pandas as pd
import random
import math

Config

In [69]:
data_path = "res/data.csv"
N = 51 #Number of node
M = 100 #Population size


Global variable

In [70]:
D = np.zeros((N,N)) # Mattrix distance betwen each node

Input

In [71]:
node_pd = pd.read_csv("../"+data_path, sep = " ", usecols=[1,2])
node = node_pd.to_numpy()
for i in range(N):
    for j in range(N):
        if (i == j) : continue
        D[i][j] = math.sqrt(sum((node[i] - node[j])*(node[i] - node[j])))

D

array([[ 0.        , 12.36931688, 19.20937271, ..., 26.40075756,
        24.20743687, 13.89244399],
       [12.36931688,  0.        , 15.29705854, ..., 21.02379604,
        13.89244399, 21.02379604],
       [19.20937271, 15.29705854,  0.        , ..., 36.22154055,
        27.29468813, 32.55764119],
       ...,
       [26.40075756, 21.02379604, 36.22154055, ...,  0.        ,
        12.04159458, 21.63330765],
       [24.20743687, 13.89244399, 27.29468813, ..., 12.04159458,
         0.        , 26.17250466],
       [13.89244399, 21.02379604, 32.55764119, ..., 21.63330765,
        26.17250466,  0.        ]])

Generate first generation

In [72]:
class Sample:

    def __init__(self):
        self.path = np.zeros(N, dtype='int') - 1
        self.weight = 0

In [73]:
def weight(path):
    global D
    weight = 0
    for i in range(1, N):
        weight = weight + D[path[i-1]][path[i]]

    weight = weight+ D[path[N-1]][path[0]]
    return weight

In [ ]:
def checkSolution(s:Sample):
    check = True
    for i in range(N):
        if i not in s.path:
            check = False
            break
    return check

: 

In [75]:
population = []
def init_population():
    global M, N, population
    population = []
    for i in range(M):
        path = random.sample(range(N),N)
        w = weight(path)
        sample = Sample()
        sample.path = path
        sample.weight = w
        population.append(sample)

Operator

In [76]:
def hybridize(s1:Sample, s2: Sample):
    s3 = Sample()
    p = random.randint(0, N-1)

    for i in range (p, N):
        s3.path[i-p] = s2.path[i]

    k = N-p
    for a in s1.path:
        if a not in s3.path:
            s3.path[k] = a
            k = k+1
            if (k >= N ) :break
    s3.weight = weight(s3.path)
    return s3


In [77]:
def mutate(s:Sample):
    s_child = Sample()
    s_child.path = s.path.copy()
    mutateCoefficient = int(N*0.1)
    count = random.randint(0, mutateCoefficient)
    for i in range(count):
        p1 = random.randint(0,N-1)
        p2 = random.randint(0,N-1)

        temp = s_child.path[p1]
        s_child.path[p1] = s_child.path[p2]
        s_child.path[p2] = temp

    s_child.weight = weight(s_child.path)
    return s_child


In [78]:
def sortPopulation():
    global population
    

GA algorithm

In [88]:
def GA(fitness_threshold, M, r, m):
    global population

    solution = Sample()
    solution.weight = 9999999

    init_population()
    population.sort(key= lambda x: x.weight)

    num_standard_sample = int((1-r)*M)
    num_hybrid_sample = M - num_standard_sample

    while (fitness_threshold > 0):
        fitness_threshold = fitness_threshold -1
        #Hybridize
        new_population = []
        new_population = population[: num_standard_sample]

        hybridize_pop = random.sample(population, num_hybrid_sample)
        random.shuffle(hybridize_pop)

        i = 0
        while i < int(num_hybrid_sample):
            s1 = hybridize(hybridize_pop[i], hybridize_pop[i+1])
            s2 = hybridize(hybridize_pop[i+1], hybridize_pop[i])
            new_population.append(s1)
            new_population.append(s2)
            i = i + 2

        for sample in new_population:
            a1 = mutate(sample)
            sample = a1

        population = new_population
        population.sort(key= lambda x: x.weight)

        if (population[0].weight < solution.weight):
            solution.path = population[0].path
            solution.weight = population[0].weight

        
    return solution


In [89]:
solution = GA(1000, M, 0.5, 0.2)

In [90]:
print(solution.path)
print(solution.weight)


[26  5 25 30 27  7 47 22  6 42 23 13 24 17  3 12 40 18 39 41 16 46 45 10
 31 21  0  1 19  2 35 34 28 20 49 33  8 15 37  4 48 38 29  9 32 44 14 43
 36 11 50]
503.00533817651944


In [91]:
solution = GA(5000, M, 0.5, 0.2)
print(solution.path)
print(solution.weight)

[28 15 10 31 50 45 11 46  3 17 16 36 14 32 44 43 41 18 39 40 12 24 13 23
 42  6 22  5 47 26  0 21 27  7 25 30  2 35 34 19  1 37  4 48  9 38 29 33
  8 49 20]
466.7547525135038


In [92]:
solution = GA(10000, M, 0.5, 0.2)
print(solution.path)
print(solution.weight)

[30 25  7 47  5 22  6 42 23 13 24 17 46 11 45 50 26  1 28 20 15 49 33 29
  8 48  9 38 32 44 14 43 41 18 39 40 12  3 16 36  4 37 10 31  0 21 19 34
 35  2 27]
446.6227640876554


In [93]:
solution = GA(100000, M, 0.5, 0.2)
print(solution.path)
print(solution.weight)

[37  4 11 46 17  3 41 18 39 40 12 24 13 23 42  6 22 47  5 26 45 50 16 36
 43 14 44 32 38  9 48  8 49 15 19 34 35  2 27 30 25  7 21  0 31 10  1 28
 20 33 29]
458.47354261492757
